# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_to_load = "../WeatherPy/output/city_data.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
dropna_weather_data.head(20)

,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,namatanai,-3.6692,152.4064,79.412,82,32,3.17,PG,1663247074
1,cape town,-33.9249,18.4241,71.384,50,100,3.58,ZA,1663247075
2,hermanus,-34.4092,19.2504,69.188,79,99,4.00,ZA,1663247076
3,toamasina,-18.1443,49.3958,73.706,68,80,4.26,MG,1663247077
4,kapaa,22.0947,-159.3246,71.726,79,40,2.57,US,1663247078
5,tuktoyaktuk,69.4454,-133.0342,36.896,71,100,8.50,CA,1663247079
6,mar del plata,-38.0055,-57.5426,57.254,77,0,5.36,AR,1663247080
7,hobart,-42.8826,147.3257,51.278,90,0,0.00,AU,1663246992
8,ribeira grande,37.8210,-25.5148,79.304,90,75,3.60,PT,1663247082
9,saskylakh,71.9543,114.1198,32.144,97,98,2.73,RU,1663247083


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
 
gmaps.configure(api_key=g_key)


locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)



In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                       dissipating=False, max_intensity=10,
                       point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]

vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]

vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]

vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]

perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]

indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,bardiyah,31.7580,25.0792,79.358,52,0,5.15,LY,1663247098
1,tsihombe,-25.3168,45.4863,78.296,58,0,7.70,MG,1663247235
2,inhambane,-23.8662,35.3827,75.488,56,0,4.71,MZ,1663247280
3,karratha,-20.7337,116.8447,74.282,46,0,4.87,AU,1663247333
4,porto velho,-8.7636,-63.8972,79.088,57,0,4.63,BR,1663247343
5,dzhusaly,45.4852,64.0819,73.832,14,0,7.72,KZ,1663247445
6,tatvan,38.5020,42.2813,75.974,14,0,3.42,TR,1663247463
7,lagoa santa,-19.6447,-43.9045,78.170,50,0,3.60,BR,1663247546
8,biscarrosse,44.3947,-1.1642,78.998,53,0,5.66,FR,1663247581
9,ozgon,40.7674,73.3031,79.718,14,0,1.61,KG,1663247598


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)


vaca_fig = gmaps.figure()


vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)


vaca_fig.add_layer(vaca_heat_layer)


vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [7]:

hotels = []


for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")


indexed_perfect_vacation["Hotel Name"] = hotels
hotel_df = indexed_perfect_vacation
indexed_perfect_vacation

,City,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
0,bardiyah,31.7580,25.0792,79.358,52,0,5.15,LY,1663247098,محلات أبناء
1,tsihombe,-25.3168,45.4863,78.296,58,0,7.70,MG,1663247235,Littoral Hôtel
2,inhambane,-23.8662,35.3827,75.488,56,0,4.71,MZ,1663247280,Oceano
3,karratha,-20.7337,116.8447,74.282,46,0,4.87,AU,1663247333,ibis Styles Karratha
4,porto velho,-8.7636,-63.8972,79.088,57,0,4.63,BR,1663247343,Ecos Hotel Conforto
5,dzhusaly,45.4852,64.0819,73.832,14,0,7.72,KZ,1663247445,Hotel Alma-Ata
6,tatvan,38.5020,42.2813,75.974,14,0,3.42,TR,1663247463,Karaman
7,lagoa santa,-19.6447,-43.9045,78.170,50,0,3.60,BR,1663247546,Hotel Lund
8,biscarrosse,44.3947,-1.1642,78.998,53,0,5.66,FR,1663247581,Saint Hubert
9,ozgon,40.7674,73.3031,79.718,14,0,1.61,KG,1663247598,Gostinitsa Uzgen


In [8]:

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:

markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)


vaca_fig

Figure(layout=FigureLayout(height='420px'))